In [35]:
import numpy as np
import pandas as pd

from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split

In [6]:
a1D = np.array([1, 2, 3, 4])
a1D.shape

(4,)

In [5]:
a2D = np.array([[1, 2], [3, 4]])
a2D.shape

(2, 2)

In [7]:
a3D = np.array([[[1, 2], [3, 4]], [[5, 6], [7, 8]]])
a3D.shape

(2, 2, 2)

# 1. Generate a large tabular dataset with 10 features, 1 regression target, 150 000 rows

In [19]:
X, y = make_regression(n_samples=150000, n_features=10, noise=0.2)

In [20]:
print(X.shape, y.shape)

(150000, 10) (150000,)


In [24]:
print(np.min(y), np.max(y))

-896.5455727802472 935.9844818086094


In [25]:
print(np.min(X), np.max(X))

-5.115738610898981 4.806065762973293


In [27]:
np.std(X, axis=1)

array([1.01464871, 0.82022551, 0.83391563, ..., 1.03507072, 1.27192982,
       0.82409952])

In [28]:
np.mean(X, axis=1)

array([ 0.22766747, -0.40564935, -0.10411321, ..., -0.23611921,
       -0.04435514,  0.27589043])

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 2. LR - Baseline

In [23]:
from sklearn.linear_model import LinearRegression

In [30]:
lr = LinearRegression()
lr.fit(lr, y_train)
print(lr.score(X_train, y_train), lr.score(X_test, y_test))

0.9999989653028651 0.9999989562275327


In [57]:
mean_absolute_percentage_error(y_train, lr.predict(X_train))

0.008881744306018605

In [58]:
mean_absolute_percentage_error(y_test, lr.predict(X_test))

0.005845866627333934

In [60]:
print('mean_absolute_percentage_error', mean_absolute_percentage_error(y_train, lr.predict(X_train)), mean_absolute_percentage_error(y_test, lr.predict(X_test)))
print('mean_absolute_error', mean_absolute_error(y_train, lr.predict(X_train)), mean_absolute_error(y_test, lr.predict(X_test)))

mean_absolute_percentage_error 0.008881744306018605 0.005845866627333934
mean_absolute_error 0.15863074922533904 0.15871495257348392


In [31]:
# y = ax1 + bx2 + cx3 + ... +  kx10 + l
# a...k  - lr.coef_
# l      - lr.intercept_
lr.coef_

array([58.55620994, 42.5633611 , 50.65246205, 43.02739377, 97.01583262,
       98.70269443, 94.08251713, 23.80257608,  1.62927657,  8.06455415])

In [32]:
lr.intercept_

4.361120083173908e-05

In [33]:
y_pred = lr.predict(X_test)

In [37]:
df = pd.DataFrame({ 'True Labels': y_test, 'Predicted Labels': y_pred})
df.head()

,True Labels,Predicted Labels
0,-108.197832,-108.089287
1,285.373432,285.349290
2,-78.163835,-78.010494
3,-45.264448,-45.312480
4,98.440974,98.051859


# DNN

In [48]:
import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader
from torch.optim import Adam

In [43]:
class DenseNeuralNetwork(nn.Module):
    def __init__(self):
        super(DenseNeuralNetwork, self).__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(10, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 1),
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits

In [44]:
dnn = DenseNeuralNetwork()

In [45]:
dnn

DenseNeuralNetwork(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=10, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=1, bias=True)
  )
)

In [52]:
criterion = nn.MSELoss()
optimizer = Adam(dnn.parameters(), lr=0.001)

x_train_tensor = torch.Tensor(X_train)
y_train_tensor = torch.Tensor(y_train)
x_test_tensor = torch.Tensor(X_test)
y_test_tensor = torch.Tensor(y_test)

train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
train_dataloader = DataLoader(train_dataset)

test_dataset = TensorDataset(x_test_tensor, y_test_tensor)
test_dataloader = DataLoader(test_dataset)

In [53]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_dataloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = dnn(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

/usr/local/lib/python3.9/site-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


[1,  2000] loss: 2759.450
[1,  4000] loss: 110.831
[1,  6000] loss: 85.168
[1,  8000] loss: 74.702
[1, 10000] loss: 71.959
[1, 12000] loss: 73.161
[1, 14000] loss: 44.374
[1, 16000] loss: 67.039
[1, 18000] loss: 51.819
[1, 20000] loss: 40.445
[1, 22000] loss: 46.328
[1, 24000] loss: 46.958
[1, 26000] loss: 36.621
[1, 28000] loss: 30.964
[1, 30000] loss: 45.107
[1, 32000] loss: 26.140
[1, 34000] loss: 31.847
[1, 36000] loss: 29.129
[1, 38000] loss: 29.866
[1, 40000] loss: 22.272
[1, 42000] loss: 44.152
[1, 44000] loss: 12.684
[1, 46000] loss: 24.905
[1, 48000] loss: 26.481
[1, 50000] loss: 22.586
[1, 52000] loss: 21.857
[1, 54000] loss: 24.740
[1, 56000] loss: 25.319
[1, 58000] loss: 20.430
[1, 60000] loss: 19.049
[1, 62000] loss: 23.321
[1, 64000] loss: 21.839
[1, 66000] loss: 25.485
[1, 68000] loss: 18.753
[1, 70000] loss: 15.752
[1, 72000] loss: 22.787
[1, 74000] loss: 32.885
[1, 76000] loss: 25.389
[1, 78000] loss: 12.424
[1, 80000] loss: 18.062
[1, 82000] loss: 15.502
[1, 84000] lo

In [59]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_absolute_error

y_train_pred = dnn(x_train_tensor).detach().numpy()
y_test_pred  = dnn(x_test_tensor).detach().numpy()

print('DNN train R2', r2_score(y_train_tensor, y_train_pred))
print('DNN test R2', r2_score(y_test_tensor, y_test_pred))
print('DNN train MAPE', mean_absolute_percentage_error(y_train_tensor, y_train_pred))
print('DNN test MAPE', mean_absolute_percentage_error(y_test_tensor, y_test_pred))
print('DNN train MAE', mean_absolute_error(y_train_tensor, y_train_pred))
print('DNN test MAE', mean_absolute_error(y_test_tensor, y_test_pred))

# LR R2 Score: 0.9999989653028651 0.9999989562275327

DNN train R2 0.9999815059273224
DNN test R2 0.9999814976258627
DNN train MAPE 0.03912664
DNN test MAPE 0.033496123
DNN train MAE 0.6591025
DNN test MAE 0.65588045


In [61]:
print('LR R2 Score', r2_score(y_train, lr.predict(X_train)), r2_score(y_test, lr.predict(X_test)))
print('LR mean_absolute_percentage_error', mean_absolute_percentage_error(y_train, lr.predict(X_train)), mean_absolute_percentage_error(y_test, lr.predict(X_test)))
print('LR mean_absolute_error', mean_absolute_error(y_train, lr.predict(X_train)), mean_absolute_error(y_test, lr.predict(X_test)))

LR R2 Score 0.9999989653028651 0.9999989562275327
LR mean_absolute_percentage_error 0.008881744306018605 0.005845866627333934
LR mean_absolute_error 0.15863074922533904 0.15871495257348392
